# Data preparation:

This notebook encompasses various essential steps to analyze the stock market data comprehensively. Initially, I gather and process the stock market data. Next, I combine this data with the data acquired in previous notebooks from FRED, topic modeling, and sentiment analysis. Additionally, I integrate another valuable data source, capturing economic policy uncertainty as expressed in news and media.

The complete process will be thoroughly explained within the notebook, providing in-depth insights into each stage of the analysis.

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

import warnings

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
# Set the start and end date
start_date = '1993-01-01'
end_date = '2023-07-27'
 
# Set the ticker
sp = '^GSPC'
dji = '^DJI'
ru2000 = '^RUT' #Russel 2000
ru3000 = '^RUA' # Russel 3000 --> captures 98% of the stock market
 
# Get the data
def stock_data(ticker, start=start_date, end=end_date):
    return yf.download(ticker, start, end)

sp = stock_data(sp)
dji = stock_data(dji)
ru2000 = stock_data(ru2000)
ru3000 = stock_data(ru3000)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


the `sp` dataset provides information about the S&P 500 index's price movements and trading activity from 1993 to 2023. The columns contain data related to opening price, highest price, lowest price, closing price, adjusted closing price, and trading volume.


In [4]:
sp.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-04,435.700012,437.320007,434.480011,435.380005,435.380005,201210000
1993-01-05,435.380005,435.399994,433.549988,434.339996,434.339996,240350000
1993-01-06,434.339996,435.170013,432.519989,434.519989,434.519989,295240000
1993-01-07,434.519989,435.459991,429.760010,430.730011,430.730011,304850000
1993-01-08,430.730011,430.730011,426.880005,429.049988,429.049988,263470000


In [5]:
# Removing columns that are not needed in the analysis
sp.drop(columns=['High', 'Low', 'Close', 'Open'], inplace=True)

In [6]:
# Calculate the percentage change in the adjusted closing price and store it in a new column 'pct_change_adj_close'.
# 'sp['Adj Close']' represents the column containing the adjusted closing prices of the financial instrument.
sp['pct_change_adj_close'] = sp['Adj Close'].pct_change() * 100

# Calculate the percentage change in the trading volume and store it in a new column 'pct_change_volume'.

sp['pct_change_volume'] = sp['Volume'].pct_change() * 100
sp.head()

,Adj Close,Volume,pct_change_adj_close,pct_change_volume
Date,,,,
1993-01-04,435.380005,201210000,NaN,NaN
1993-01-05,434.339996,240350000,-0.238874,19.452314
1993-01-06,434.519989,295240000,0.041441,22.837529
1993-01-07,430.730011,304850000,-0.872222,3.254979
1993-01-08,429.049988,263470000,-0.390041,-13.573889


In [7]:
# Create a new column 'lag_pct_change_adj_close' and shift the values of 'pct_change_adj_close' one step forward.
# The 'shift()' function moves the values one position backward, effectively creating a lagged version of the column.
# The 'lag_pct_change_adj_close' column will now contain the percentage change values of the adjusted closing price
# for the previous trading day.
sp['lag_pct_change_adj_close'] = sp['pct_change_adj_close'].shift()

# Create a new column 'lag_pct_change_volume' and shift the values of 'pct_change_volume' one step forward.
sp['lag_pct_change_volume'] = sp['pct_change_volume'].shift()

sp.head()

,Adj Close,Volume,pct_change_adj_close,pct_change_volume,lag_pct_change_adj_close,lag_pct_change_volume
Date,,,,,,
1993-01-04,435.380005,201210000,NaN,NaN,NaN,NaN
1993-01-05,434.339996,240350000,-0.238874,19.452314,NaN,NaN
1993-01-06,434.519989,295240000,0.041441,22.837529,-0.238874,19.452314
1993-01-07,430.730011,304850000,-0.872222,3.254979,0.041441,22.837529
1993-01-08,429.049988,263470000,-0.390041,-13.573889,-0.872222,3.254979


In [8]:
# Create a new column 'lag_ma5_pct_change_adj_close' and calculate the rolling 5-day moving average of 'lag_pct_change_adj_close'.
# The 'rolling(5)' function creates a rolling window of size 5 days, and 'mean()' calculates the average of the values within the window.
# The 'lag_ma5_pct_change_adj_close' column will now contain the 5-day moving average of the percentage change in the adjusted closing price.
sp['lag_ma5_pct_change_adj_close'] = sp['lag_pct_change_adj_close'].rolling(5).mean()
sp.head(10) 

,Adj Close,Volume,pct_change_adj_close,pct_change_volume,lag_pct_change_adj_close,lag_pct_change_volume,lag_ma5_pct_change_adj_close
Date,,,,,,,
1993-01-04,435.380005,201210000,NaN,NaN,NaN,NaN,NaN
1993-01-05,434.339996,240350000,-0.238874,19.452314,NaN,NaN,NaN
1993-01-06,434.519989,295240000,0.041441,22.837529,-0.238874,19.452314,NaN
1993-01-07,430.730011,304850000,-0.872222,3.254979,0.041441,22.837529,NaN
1993-01-08,429.049988,263470000,-0.390041,-13.573889,-0.872222,3.254979,NaN
1993-01-11,430.950012,217150000,0.442845,-17.580749,-0.390041,-13.573889,NaN
1993-01-12,431.040009,239410000,0.020883,10.250979,0.442845,-17.580749,-0.203370
1993-01-13,433.029999,245360000,0.461672,2.485276,0.020883,10.250979,-0.151419
1993-01-14,435.940002,281040000,0.672010,14.541898,0.461672,2.485276,-0.067373


Next, I import the FOMC minutes data, which contains the topic modeling data and the sentiment analyses data.

In [9]:
fomc_topic = pd.read_pickle('../data/fomc_topic_modeling.pkl')

fomc_topic.drop(columns=['minutes_paragraphs', 'paragraphs_length', 'minutes_text',
       'text_length', 'topic_weight', 'topic_score', 'doc2bow'], inplace=True)

fomc_topic.head()

,topic 1,consumption,foreign_exchange_rate,inflation,financial_market,topic 6
1993-02-03,0.217142,0.222028,0.071233,0.354325,0.077838,0.057434
1993-03-23,0.052435,0.337770,0.102628,0.422744,0.049573,0.034851
1993-05-18,0.044406,0.325946,0.120336,0.428469,0.042354,0.038488
1993-07-07,0.031498,0.281643,0.069809,0.522031,0.053767,0.041252
1993-08-17,0.036917,0.411438,0.112277,0.312171,0.081001,0.046197


In [10]:
fomc_laughran_mcdonald = pd.read_pickle('../data/fomc_raw_laughran_mcdonald_sentiment.pkl')
fomc_laughran_mcdonald = fomc_laughran_mcdonald[['sentiment_mc']]
fomc_laughran_mcdonald.head()

,sentiment_mc
1993-02-03,1.30719
1993-03-23,1.11151
1993-05-18,1.486831
1993-07-07,2.200566
1993-08-17,2.022472


In [11]:
fomc_zstc_sentiment = pd.read_pickle('../data/fomc_raw_zstc_sentiment.pkl')
fomc_zstc_sentiment.drop(columns='fomc_text', inplace=True)
fomc_zstc_sentiment.head()

,sentiment_zstc
date,
1993-02-03,0.703742
1993-03-23,0.402003
1993-05-18,0.270261
1993-07-07,0.571819
1993-08-17,0.412568


In the next step, I combine the three dataframes, merging them into a single comprehensive dataset. This merged dataset will contain all the information gathered from the various FOMC meetings.

In [12]:
fomc = fomc_topic.merge(fomc_laughran_mcdonald, how = 'inner', left_index=True, right_index=True) \
                 .merge(fomc_zstc_sentiment, how = 'inner', left_index=True, right_index=True)

Since the stock market operates only on weekdays, it's essential to check if all FOMC meetings happened during weekdays too. To do this, I add a new column called "dayofweek" to indicate the meeting day.

After reviewing the data, I find that one of the meetings took place on a Sunday, which could cause us to miss valuable information. To avoid this, I move the data from that Sunday meeting to the following Monday. By doing so, we ensure that we don't lose any important data, and all meetings align with the market's weekday schedule.

In [13]:
# The 'strftime('%A')' function is used to convert the date from the index to the full weekday name.
# For example, if the index contains '2023-07-30', the 'dayofweek' column for that row will have the value 'Saturday'.
fomc['dayofweek'] = fomc.index.strftime('%A')

fomc['dayofweek'].value_counts()


dayofweek
Wednesday    135
Tuesday       99
Thursday       8
Friday         1
Sunday         1
Name: count, dtype: int64

In [14]:
fomc.reset_index(names='date', inplace=True)

In [15]:
sunday_row = fomc['dayofweek'] =='Sunday'
fomc[sunday_row]

,date,topic 1,consumption,foreign_exchange_rate,inflation,financial_market,topic 6,sentiment_mc,sentiment_zstc,dayofweek
217,2020-03-15,0.134648,0.212484,0.138263,0.337477,0.134108,0.043019,-3.864534,0.327511,Sunday


In [16]:
# For those meetings on Sundays, add one day to their 'date' values to move them to the following Monday.
# The 'pd.Timedelta(1, "d")' represents a time duration of one day, and it is added to the 'date' column.
fomc.loc[sunday_row, 'date'] = fomc.loc[sunday_row, 'date'] + pd.Timedelta(1, "d")

fomc.loc[sunday_row, 'date']

217   2020-03-16
Name: date, dtype: datetime64[ns]

In [17]:
fomc.loc[sunday_row, 'dayofweek'] = 'Monday'

In [18]:
fomc['dayofweek'].value_counts()

dayofweek
Wednesday    135
Tuesday       99
Thursday       8
Friday         1
Monday         1
Name: count, dtype: int64

In [19]:
fomc.set_index('date', inplace=True)

Next, I merge sp data with fomc and drop `dayofweek` column.

In [20]:
# Perform a left merge between the DataFrames 'sp' and 'fomc_topic' based on their index values.
data = pd.merge(sp, fomc, how='left', left_index=True, right_index=True)

# Drop the 'dayofweek' column from the merged DataFrame 'data'.
data = data.drop(columns='dayofweek')

data.head()

,Adj Close,Volume,pct_change_adj_close,pct_change_volume,lag_pct_change_adj_close,lag_pct_change_volume,lag_ma5_pct_change_adj_close,topic 1,consumption,foreign_exchange_rate,inflation,financial_market,topic 6,sentiment_mc,sentiment_zstc
Date,,,,,,,,,,,,,,,
1993-01-04,435.380005,201210000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-05,434.339996,240350000,-0.238874,19.452314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-06,434.519989,295240000,0.041441,22.837529,-0.238874,19.452314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-07,430.730011,304850000,-0.872222,3.254979,0.041441,22.837529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-08,429.049988,263470000,-0.390041,-13.573889,-0.872222,3.254979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
print(sp.shape)
print(fomc_topic.shape)

(7696, 7)
(244, 6)


In [22]:
data.notna().sum()

Adj Close                       7696
Volume                          7696
pct_change_adj_close            7695
pct_change_volume               7695
lag_pct_change_adj_close        7694
lag_pct_change_volume           7694
lag_ma5_pct_change_adj_close    7690
topic 1                          244
consumption                      244
foreign_exchange_rate            244
inflation                        244
financial_market                 244
topic 6                          244
sentiment_mc                     244
sentiment_zstc                   244
dtype: int64

> A possible concern is whether FOMC meetings might fall on weekdays that are holidays, which is unlikely but still important to consider. However, upon reviewing the data, we find that all 244 meeting dates are included in the final dataset.

In [23]:
# Create a new DataFrame 'data_filled' by forward-filling missing values in the original DataFrame 'data'.
data_filled = data.ffill()

In [24]:
data_filled.head(5)

,Adj Close,Volume,pct_change_adj_close,pct_change_volume,lag_pct_change_adj_close,lag_pct_change_volume,lag_ma5_pct_change_adj_close,topic 1,consumption,foreign_exchange_rate,inflation,financial_market,topic 6,sentiment_mc,sentiment_zstc
Date,,,,,,,,,,,,,,,
1993-01-04,435.380005,201210000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-05,434.339996,240350000,-0.238874,19.452314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-06,434.519989,295240000,0.041441,22.837529,-0.238874,19.452314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-07,430.730011,304850000,-0.872222,3.254979,0.041441,22.837529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-08,429.049988,263470000,-0.390041,-13.573889,-0.872222,3.254979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next, Iwe add the FRED data to our existing dataset.

In [25]:
fred = pd.read_pickle('../data/fred.pkl')
fred.head(3)

,ffr,unemployment_rate,median_cpi,retail_sales,10_year_treasury_yeild,10year_3month_yield_spread,vix,us_china_exchange_rate,us_japan_exchange_rate,inflation,growth
date,,,,,,,,,,,
1993-01-01,2.66,7.3,3.442924,175108.0,6.6,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-02,2.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-03,2.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


FRED data contains various indicators with different frequency levels. While some indicators have daily data, others might have quarterly or monthly data, which can fall on weekends. To ensure that no data points are missed, I apply two methods from the pandas package: `resample` and `ffill`.

Using the `resample` method, I create a complete sample that covers all days from 1993 to 2023. Then, the `ffill` method is used to forward-fill any missing data points, ensuring that all relevant information is included in the final dataset. 

In [26]:
fred.describe()

,ffr,unemployment_rate,median_cpi,retail_sales,10_year_treasury_yeild,10year_3month_yield_spread,vix,us_china_exchange_rate,us_japan_exchange_rate,inflation,growth
count,10208.000000,366.000000,366.000000,365.000000,365.00000,7649.000000,7706.000000,7643.000000,7668.000000,365.000000,121.000000
mean,2.478556,5.710383,2.747473,369190.167123,3.90074,1.547090,19.704661,7.335381,109.208848,0.207468,0.610526
std,2.209207,1.786939,1.272248,127066.737926,1.73790,1.175057,8.177889,0.890454,13.812318,0.279168,1.218524
min,0.040000,3.400000,-0.286064,172306.000000,0.62000,-1.890000,9.140000,5.707600,75.720000,-1.770548,-8.484336
25%,0.150000,4.400000,2.100500,275192.000000,2.40000,0.640000,13.690000,6.529350,102.697500,0.068489,0.364447
50%,1.890000,5.400000,2.542004,357331.000000,3.85000,1.550000,17.920000,7.029000,109.500000,0.203804,0.638435
75%,4.910000,6.300000,3.162344,448091.000000,5.24000,2.530000,23.277500,8.277200,117.967500,0.327690,0.943827
max,7.800000,14.700000,8.463917,692501.000000,7.96000,3.850000,82.690000,8.740900,149.820000,1.376849,7.854483


In [27]:
(fred['10_year_treasury_yeild']==0).sum()

0

Fortunately, there are no values equal to zero in the FRED dataset. Therefore, we can create a complete sample spanning from 1993 to 2023 by utilizing Pandas `resample` method. This function will help us generate a dataset containing all the days within this timeframe. After creating the sample, we will replace any zero values with `NaN`  again, to maintain consistency with the rest of the data.

In [28]:
fred = fred.resample('D').sum()
fred.head(3)

,ffr,unemployment_rate,median_cpi,retail_sales,10_year_treasury_yeild,10year_3month_yield_spread,vix,us_china_exchange_rate,us_japan_exchange_rate,inflation,growth
date,,,,,,,,,,,
1993-01-01,2.66,7.3,3.442924,175108.0,6.6,0.0,0.0,0.0,0.0,0.0,0.0
1993-01-02,2.66,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1993-01-03,2.66,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
fred.replace(0, np.nan, inplace=True)
fred.head(3)

,ffr,unemployment_rate,median_cpi,retail_sales,10_year_treasury_yeild,10year_3month_yield_spread,vix,us_china_exchange_rate,us_japan_exchange_rate,inflation,growth
date,,,,,,,,,,,
1993-01-01,2.66,7.3,3.442924,175108.0,6.6,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-02,2.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-03,2.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, I apply the `ffill` method for forward filling. This process ensures that any gaps in the data are filled with the most recent available value, completing our dataset and preparing it for further analysis.

In [30]:
fred.ffill(inplace=True)

In [31]:
fred.head(3)

,ffr,unemployment_rate,median_cpi,retail_sales,10_year_treasury_yeild,10year_3month_yield_spread,vix,us_china_exchange_rate,us_japan_exchange_rate,inflation,growth
date,,,,,,,,,,,
1993-01-01,2.66,7.3,3.442924,175108.0,6.6,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-02,2.66,7.3,3.442924,175108.0,6.6,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-03,2.66,7.3,3.442924,175108.0,6.6,NaN,NaN,NaN,NaN,NaN,NaN


At last, it is time to introduce and include the final indicator to the dataset, Economic Policy Uncertainty. The Economic Policy Uncertainty (EPU) Index is an economic indicator that measures the level of uncertainty in a country's economy arising from policy-related factors. It is designed to capture the extent to which economic policymakers (e.g., governments, central banks) are creating uncertainty through their decisions or the changing policy environment. The index helps in quantifying and monitoring economic uncertainty, which can have significant implications for businesses, investors, and policymakers.

The EPU Index composed of three types of underlying components. The first component quantifies newspaper coverage of policy-related economic uncertainty from ten large newspapers in the United States. The second component involves the number of federal tax code provisions set to expire, and the third component utilizes the dispersion between individual forecasters' predictions about future macroeconomic variables from the Federal Reserve Bank of Philadelphia's Survey of Professional Forecasters. These components collectively form the policy-related uncertainty index. ([source](http://www.policyuncertainty.com/index.html))

In [32]:
epu = pd.read_excel('../data/us_policy_uncertainty.xlsx', sheet_name='Main Index')
epu = epu.iloc[0:-1, :]
epu.head()

,Year,Month,Three_Component_Index,News_Based_Policy_Uncert_Index
0,1985,1.0,125.224739,103.748802
1,1985,2.0,99.020813,78.313202
2,1985,3.0,112.190506,100.761482
3,1985,4.0,102.811325,84.778870
4,1985,5.0,120.082726,98.053658


In [33]:
epu.dtypes

Year                               object
Month                             float64
Three_Component_Index             float64
News_Based_Policy_Uncert_Index    float64
dtype: object

In [34]:
# Combine the 'Year' and 'Month' columns to create a new 'date' column in the DataFrame.
# The 'date' column will represent the first day of each month in the format 'YYYY-MM-1'.
epu['date'] = epu['Year'] + '-' + epu['Month'].astype(int).astype(str) + '-1'

# Convert the 'date' column to the datetime data type using the 'pd.to_datetime()' function.
epu['date'] = pd.to_datetime(epu['date'])

# Set the 'date' column as the new index of the DataFrame 'us_policy_uncertainty'.
epu.set_index('date', inplace=True)

# Drop the 'Year' and 'Month' columns from the DataFrame 'us_policy_uncertainty'.
epu.drop(columns=['Year', 'Month'], inplace=True)
epu.head(3)

,Three_Component_Index,News_Based_Policy_Uncert_Index
date,,
1985-01-01,125.224739,103.748802
1985-02-01,99.020813,78.313202
1985-03-01,112.190506,100.761482


EPU, being recorded monthly, requires a similar cleaning approach as I used for FRED data. To achieve this, I will utilize the `resample` and `ffill` methods once again. By resampling the data, I create a complete daily sample that covers the entire duration. Then, the "ffill" method will be applied to fill in missing values. This process allows us to effectively handle any gaps or irregularities in the EPU data.

In [35]:
epu = epu.resample('D').sum()
epu.head(3)

,Three_Component_Index,News_Based_Policy_Uncert_Index
date,,
1985-01-01,125.224739,103.748802
1985-01-02,0.000000,0.000000
1985-01-03,0.000000,0.000000


In [36]:
epu.replace(0, np.nan, inplace=True)
epu.head(3)

,Three_Component_Index,News_Based_Policy_Uncert_Index
date,,
1985-01-01,125.224739,103.748802
1985-01-02,NaN,NaN
1985-01-03,NaN,NaN


In [37]:
epu.ffill(inplace=True)
epu.head(3)

,Three_Component_Index,News_Based_Policy_Uncert_Index
date,,
1985-01-01,125.224739,103.748802
1985-01-02,125.224739,103.748802
1985-01-03,125.224739,103.748802


> After importing and processing the economic policy uncertainty data from its original source, I made an interesting observation. It turns out that the index is also available in FRED, and it can be accessed using the series ID USEPUINDXD. 

Finally, I merge the data from `FRED` and the economic policy uncertainty (`EPU`) with the main dataframe, which is called `data`.  Additionally, I apply the "ffill" method once more to fill in missing values in the FOMC variables within the final sample. After that, all remaining missing values are removed.

In [38]:
data = pd.merge(data, fred, how='left', left_index=True, right_index=True) \
         .merge(epu, how='left', left_index=True, right_index=True)
data.ffill(inplace=True)
data.dropna(inplace=True)

In [39]:
data.head(3) 

,Adj Close,Volume,pct_change_adj_close,pct_change_volume,lag_pct_change_adj_close,lag_pct_change_volume,lag_ma5_pct_change_adj_close,topic 1,consumption,foreign_exchange_rate,inflation_x,financial_market,topic 6,sentiment_mc,sentiment_zstc,ffr,unemployment_rate,median_cpi,retail_sales,10_year_treasury_yeild,10year_3month_yield_spread,vix,us_china_exchange_rate,us_japan_exchange_rate,inflation_y,growth,Three_Component_Index,News_Based_Policy_Uncert_Index
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1993-04-01,450.299988,234530000,-0.303324,-15.996275,-0.066373,20.762144,0.161433,0.052435,0.33777,0.102628,0.422744,0.049573,0.034851,1.11151,0.402003,3.31,7.1,4.09652,176749.0,5.97,3.10,13.02,5.7333,114.1,0.348918,0.582113,109.256836,89.651123
1993-04-02,441.390015,323330000,-1.978675,37.862960,-0.303324,-15.996275,-0.024659,0.052435,0.33777,0.102628,0.422744,0.049573,0.034851,1.11151,0.402003,3.13,7.1,4.09652,176749.0,5.97,3.19,14.50,5.7333,114.2,0.348918,0.582113,109.256836,89.651123
1993-04-05,442.290009,296080000,0.203900,-8.427922,-1.978675,37.862960,-0.282885,0.052435,0.33777,0.102628,0.422744,0.049573,0.034851,1.11151,0.402003,3.15,7.1,4.09652,176749.0,5.97,3.16,14.12,5.7333,113.6,0.348918,0.582113,109.256836,89.651123


The last set of data preparation steps are to create a dummy variable that indicate whether the stock prices has increases or not in each trading day. this variable will serve as the target variable in the analysis. Then, I remove, Volume and pct_change_volume from the dataset to avoid inputing forward looking data to the model.

The final steps of data preparation involve creating a dummy variable to indicate whether the stock prices have increased or not on each trading day. This variable will serve as the target variable for the analysis. Additionally, to avoid inputting forward-looking data into the model, I remove the `Volume` and `pct_change_volume` features from the dataset. By doing so, I ensure that the model uses only historical information for analysis and predictions, maintaining the integrity of the analysis and avoiding any data leakage issues. 

In [40]:
data['price_increase'] = np.where(data['pct_change_adj_close']>=0, 1, 0)

In [41]:
data.drop(columns=['Volume', 'pct_change_volume'], inplace=True)

In [42]:
data.head(3)

,Adj Close,pct_change_adj_close,lag_pct_change_adj_close,lag_pct_change_volume,lag_ma5_pct_change_adj_close,topic 1,consumption,foreign_exchange_rate,inflation_x,financial_market,topic 6,sentiment_mc,sentiment_zstc,ffr,unemployment_rate,median_cpi,retail_sales,10_year_treasury_yeild,10year_3month_yield_spread,vix,us_china_exchange_rate,us_japan_exchange_rate,inflation_y,growth,Three_Component_Index,News_Based_Policy_Uncert_Index,price_increase
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,
1993-04-01,450.299988,-0.303324,-0.066373,20.762144,0.161433,0.052435,0.33777,0.102628,0.422744,0.049573,0.034851,1.11151,0.402003,3.31,7.1,4.09652,176749.0,5.97,3.10,13.02,5.7333,114.1,0.348918,0.582113,109.256836,89.651123,0
1993-04-02,441.390015,-1.978675,-0.303324,-15.996275,-0.024659,0.052435,0.33777,0.102628,0.422744,0.049573,0.034851,1.11151,0.402003,3.13,7.1,4.09652,176749.0,5.97,3.19,14.50,5.7333,114.2,0.348918,0.582113,109.256836,89.651123,0
1993-04-05,442.290009,0.203900,-1.978675,37.862960,-0.282885,0.052435,0.33777,0.102628,0.422744,0.049573,0.034851,1.11151,0.402003,3.15,7.1,4.09652,176749.0,5.97,3.16,14.12,5.7333,113.6,0.348918,0.582113,109.256836,89.651123,1


In [43]:
data.to_pickle('../data/full_data.pkl')